In [5]:
import os
import pandas as pd
import re
print(os.getcwd())

c:\Users\Joshualevy\Documents\ioCapture\01_Database_Construction\papers


In [3]:
fuzzy_reports_paths_lists = []
fuzzy_matched_csvs_lists = []
fuzzy_results_folders_path = '../../00_Scraping/econlit_scopus_matching_out/'
for current_folder,child_folders,current_folder_files in os.walk(fuzzy_results_folders_path):
    for file in current_folder_files:
        if re.search(r'_fuzzy_matching_report\.txt', file, re.I):
            new_report_path = current_folder + '/' + file
            fuzzy_reports_paths_lists.append(new_report_path)

        if re.search(r'_fuzzy_matches\.csv', file, re.I):
            fuzzy_matches_path = current_folder + '/' + file
            fuzzy_matched_csvs_lists.append(fuzzy_matches_path)




In [6]:
fuzzy_reports_df = pd.DataFrame()

for path in fuzzy_reports_paths_lists:
    print(path)
    pub_code = re.search(r'([A-Z]{3})_fuzzy_matching_report\.txt', path)[1]
    print(pub_code)

    fuzzy_report_patterns_dict = {
        'scopus_obs' : r'Original ' + pub_code + r' Scopus-collected observations: (\d{3,5})',
        'econlit_obs' : r'Original ' + pub_code + r' EconLit-collected observations: (\d{3,5})',
        'naive_match_obs' : r'Number of ' + pub_code + r' naively-matched observations: (\d{3,5})',
        'fuzzy_match_obs' : r'Number of ' + pub_code + r' fuzzy-matched observations \d+ \((\d{2,5}) added\)',
        'remaining_scopus_obs' : r'Number of ' + pub_code + r' post-fuzzy-match Scopus-only remainder observations: (\d{1,5})',
        'remaining_econlit_obs' : r'Number of ' + pub_code + r' post-fuzzy-match EconLit-only remainder observations: (\d{1,5})',
    }


    temp_df = pd.DataFrame({
        'pub_code' : pub_code,
        'scopus_obs' : [None],
        'econlit_obs' : [None],
        'naive_match_obs' :  [None],
        'fuzzy_match_obs' : [None],
        'remaining_scopus_obs' : [None],
        'remaining_econlit_obs' : [None],

    })

    fuzzy_report_file = open(path, 'r')
    fuzzy_report_lines = fuzzy_report_file.read()

    for column in fuzzy_report_patterns_dict.keys():
        pattern = fuzzy_report_patterns_dict.get(column)
        patterns_value = re.search(pattern, fuzzy_report_lines)[1]
        print('{}: {}'.format(column, patterns_value))
        temp_df.loc[0, column] = patterns_value

    fuzzy_reports_df = pd.concat([fuzzy_reports_df, temp_df], ignore_index=True)


fuzzy_reports_df.to_csv('fuzzy_match_results.csv')


../../00_Scraping/econlit_scopus_matching_out/AER_fuzzy_results/AER_fuzzy_matching_report.txt
AER
scopus_obs: 4814
econlit_obs: 5679
naive_match_obs: 4449
fuzzy_match_obs: 354
remaining_scopus_obs: 11
remaining_econlit_obs: 876
../../00_Scraping/econlit_scopus_matching_out/ATB_fuzzy_results/ATB_fuzzy_matching_report.txt
ATB
scopus_obs: 725
econlit_obs: 890
naive_match_obs: 547
fuzzy_match_obs: 132
remaining_scopus_obs: 53
remaining_econlit_obs: 217
../../00_Scraping/econlit_scopus_matching_out/ECA_fuzzy_results/ECA_fuzzy_matching_report.txt
ECA
scopus_obs: 1574
econlit_obs: 1918
naive_match_obs: 1513
fuzzy_match_obs: 54
remaining_scopus_obs: 7
remaining_econlit_obs: 351
../../00_Scraping/econlit_scopus_matching_out/JEM_fuzzy_results/JEM_fuzzy_matching_report.txt
JEM
scopus_obs: 739
econlit_obs: 960
naive_match_obs: 606
fuzzy_match_obs: 101
remaining_scopus_obs: 32
remaining_econlit_obs: 253
../../00_Scraping/econlit_scopus_matching_out/JFE_fuzzy_results/JFE_fuzzy_matching_report.txt
JF